# 🧪 Fine-Tuning a Small Language Model Using LoRA

This tutorial walks you through the process of fine-tuning a causal language model (e.g., distilgpt2) using the LoRA (Low-Rank Adaptation) method with Hugging Face Transformers and PEFT.

## 📦 Requirements

Make sure you have the following Python libraries installed:

In [1]:
#!pip install transformers datasets peft accelerate torch

## 🔢 Step 1: Load and Prepare Your Dataset

First, we load the dataset from a .jsonl file and convert each entry into a single string with both instruction and output.

In [2]:
from datasets import load_dataset

def convert_to_hf_format(example):
    return {
        "text": f"Instruction: {example['instruction']}\nOutput: {example['output']}"
    }

dataset = load_dataset('json', data_files='dataset.jsonl')
dataset = dataset.map(convert_to_hf_format)
dataset = dataset["train"].train_test_split(test_size=0.2)

c:\Users\thomm\OneDrive\Desktop\Repositorios\EdgeAI\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 🤖 Step 2: Load the Pretrained Model and Tokenizer

Use Hugging Face to load the model and tokenizer. If the tokenizer lacks a pad_token, we use the eos_token.

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

## ✂️ Step 3: Tokenize the Dataset

Tokenize the combined "instruction + output" text using a fixed max_length.

In [4]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 2/2 [00:00<00:00, 59.52 examples/s]


## 🔧 Step 4: Prepare Model for LoRA Training

Enable LoRA fine-tuning and quantization-aware training using PEFT:

In [5]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)


## 🧩 Step 5: Configure and Inject LoRA

Set up LoRA with desired hyperparameters and specify which modules to adapt:

In [6]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["c_attn", "c_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    inference_mode=False
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 811,008 || all params: 82,723,584 || trainable%: 0.9804


c:\Users\thomm\OneDrive\Desktop\Repositorios\EdgeAI\.venv\lib\site-packages\peft\tuners\lora\layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


✅ Only a small subset of parameters will be trained.

## 🧱 Step 6: Define Data Collator

We use a data collator for causal language modeling without masked language modeling (MLM).

In [7]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)


## ⚙️ Step 7: Set Training Arguments

Configure training parameters like learning rate, batch size, and evaluation strategy.

In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    eval_strategy="steps",
    eval_steps=100,
    learning_rate=1e-3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=100,
    weight_decay=0.01,
    logging_steps=10,
    save_strategy="steps",
    save_steps=100,
    load_best_model_at_end=True,
    report_to="none",
)


## 🏋️ Step 8: Initialize Trainer

Bring everything together with the Hugging Face Trainer.

In [9]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
)


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


## 🚀 Step 9: Train the Model

Start the training process:

In [10]:
trainer.train()

c:\Users\thomm\OneDrive\Desktop\Repositorios\EdgeAI\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
100,0.258900,6.514019
200,0.097400,6.728522


c:\Users\thomm\OneDrive\Desktop\Repositorios\EdgeAI\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=200, training_loss=0.6558601385354996, metrics={'train_runtime': 908.8732, 'train_samples_per_second': 0.88, 'train_steps_per_second': 0.22, 'total_flos': 26627958374400.0, 'train_loss': 0.6558601385354996, 'epoch': 100.0})

## 💾 Step 10: Save Your Model and Tokenizer

Save the fine-tuned model

In [11]:
model.save_pretrained("lora_finetuned_model")
tokenizer.save_pretrained('distilgpt2_tokenizer')

('distilgpt2_tokenizer\\tokenizer_config.json',
 'distilgpt2_tokenizer\\special_tokens_map.json',
 'distilgpt2_tokenizer\\vocab.json',
 'distilgpt2_tokenizer\\merges.txt',
 'distilgpt2_tokenizer\\added_tokens.json',
 'distilgpt2_tokenizer\\tokenizer.json')

## 💻 Step 11: Test Your Model and Tokenizer

Define a simple generation function to test.

In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("lora_finetuned_model")
tokenizer = AutoTokenizer.from_pretrained("distilgpt2_tokenizer")

In [13]:
def generate_response(instruction, input_text=""):
    prompt = f"Instruction: {instruction}\nOutput:" if not input_text else f"Instruction: {instruction}\nInput: {input_text}\nOutput:"
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=50,
        pad_token_id=tokenizer.eos_token_id,
        temperature=0.7
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True).split("Output:")[-1].strip()

In [14]:
# Example Test
print("Test response:", generate_response("How do I activate cruise control?"))

c:\Users\thomm\OneDrive\Desktop\Repositorios\EdgeAI\.venv\lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Test response: To use cruise control in a 2023 Subaru Outback:
1. Press the 'CRUISE' button on the steering wheel
2. Accelerate to desired speed (above 25 mph)
3. Press 'SET' to engage
